# Classificação de imagens de tomografia computadorizada de próstata

## Instalações

In [1]:
import sys

In [2]:
!{sys.executable} -m pip install pydicom

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
!{sys.executable} -m pip install pypng

  Running setup.py bdist_wheel for pypng: started
  Running setup.py bdist_wheel for pypng: finished with status 'done'
  Stored in directory: C:\Users\gabrielle\AppData\Local\pip\Cache\wheels\ed\30\6c\21dd4d267f47ea09fb57881b30fe16f7231b71bd26dea38afc
Successfully built pypng


You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
!{sys.executable} -m pip install keras

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
!{sys.executable} -m pip install tensorflow

  Running setup.py bdist_wheel for gast: started
  Running setup.py bdist_wheel for gast: finished with status 'done'
  Stored in directory: C:\Users\gabrielle\AppData\Local\pip\Cache\wheels\9a\1f\0e\3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
  Running setup.py bdist_wheel for termcolor: started
  Running setup.py bdist_wheel for termcolor: finished with status 'done'
  Stored in directory: C:\Users\gabrielle\AppData\Local\pip\Cache\wheels\7c\06\54\bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for absl-py: started
  Running setup.py bdist_wheel for absl-py: finished with status 'done'
  Stored in directory: C:\Users\gabrielle\AppData\Local\pip\Cache\wheels\78\a3\a3\689120b95c26b9a21be6584b4b482b0fda0a1b60efd30af558
Successfully built gast termcolor absl-py
  Found existing installation: numpy 1.13.1
    Uninstalling numpy-1.13.1:
      Successfully uninstalled numpy-1.13.1


You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Definição de paths e arquivos treino / teste

In [6]:
training_data = 'imagens/treinamento'
validation_data = 'imagens/validacao'

In [21]:
import pylab
import pydicom as dicom
import glob
import png
import os

#Transforma as imagens DICOM em PNG

diretorio_list = ['imagens/treinamento/true/', 'imagens/validacao/true/',
                  'imagens/treinamento/false/', 'imagens/validacao/false/']
for diretorio in diretorio_list:
    for img in glob.glob(diretorio + '*.dcm'):
          mri_file = open(img, 'rb')
          imagem = dicom.read_file(mri_file)
          mri_file.close()

          shape = imagem.pixel_array.shape

          image_2d = []
          max_val = 0
          for row in imagem.pixel_array:
              pixels = []
              for col in row:
                  pixels.append(col)
                  if col > max_val: max_val = col
              image_2d.append(pixels)

          image_2d_scaled = []
          for row in image_2d:
              row_scaled = []
              for col in row:
                  col_scaled = int((float(col) / float(max_val)) * 255.0)
                  row_scaled.append(col_scaled)
              image_2d_scaled.append(row_scaled)

          png_nome = os.path.basename(img)[:-3] + 'png'
          print(png_nome)
          png_file = open(diretorio+png_nome, 'wb')
          w = png.Writer(shape[1], shape[0], greyscale=True)
          w.write(png_file, image_2d_scaled)
          png_file.close()


0000-12-000007.png
0002-12-000007.png
0005-12-000007.png
0014-12-000007.png
0015-12-000007.png
0018-12-000007.png
0019-12-000007.png
0026-12-000007.png
0028-12-000007.png
0031-12-000007.png
0035-12-000007.png
0037-12-000007.png
0040-12-000007.png
0041-12-000007.png
0046-12-000007.png
0054-12-000007.png
0055-12-000007.png
0067-12-000007.png
0071-12-000007.png
0077-12-000007.png
0078-12-000007.png
0082-12-000007.png
0092-12-000007.png
0094-12-000007.png
0095-12-000007.png
0097-12-000007.png
0099-12-000007.png
0102-12-000007.png
0103-12-000007.png
0104-12-000007.png
0105-12-000007.png
0106-12-000007.png
0111-12-000007.png
0114-12-000007.png
0115-12-000007.png
0117-12-000007.png
0118-12-000007.png
0119-12-000007.png
0121-12-000007.png
0123-12-000007.png
0125-12-000007.png
0126-12-000007.png
0127-12-000007.png
0128-12-000007.png
0129-12-000007.png
0131-12-000007.png
0139-12-000007.png
0143-12-000007.png
0164-12-000007.png
0169-12-000007.png
0179-12-000007.png
0181-12-000007.png
0183-12-0000

In [22]:
import keras
import numpy as np

import os
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

In [23]:
img_width, img_height = 256, 256

# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        training_data,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')


Found 182 images belonging to 2 classes.
Found 27 images belonging to 2 classes.


In [24]:
# Initialising the CNN
model = Sequential()

# Convolution
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))

# Pooling
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening
model.add(Flatten())

# Full connection
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, activation = 'sigmoid'))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(256, 256,...)`
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  if sys.path[0] == '':


## Treinamento

In [25]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [26]:
#Definição de variáveis de treinamento

nb_epoch  = 10
nb_train_samples = 160
nb_validation_samples = 24

model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)
model.save_weights('modelos/checkpoint.h5')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=10, epochs=10, validation_steps=24)`
  if sys.path[0] == '':


Epoch 1/10
10/10 [==============================] - 17s 2s/step - loss: 3.8976 - acc: 0.7125 - val_loss: 4.1788 - val_acc: 0.7407
Epoch 2/10
10/10 [==============================] - 13s 1s/step - loss: 4.2336 - acc: 0.7373 - val_loss: 4.1788 - val_acc: 0.7407
Epoch 3/10
10/10 [==============================] - 15s 2s/step - loss: 4.6340 - acc: 0.7125 - val_loss: 4.1788 - val_acc: 0.7407
Epoch 4/10
10/10 [==============================] - 14s 1s/step - loss: 3.5472 - acc: 0.7799 - val_loss: 4.1788 - val_acc: 0.7407
Epoch 5/10
10/10 [==============================] - 14s 1s/step - loss: 5.1199 - acc: 0.6823 - val_loss: 4.1788 - val_acc: 0.7407
Epoch 6/10
10/10 [==============================] - 14s 1s/step - loss: 4.0051 - acc: 0.7515 - val_loss: 4.1788 - val_acc: 0.7407
Epoch 7/10
10/10 [==============================] - 15s 1s/step - loss: 4.1303 - acc: 0.7438 - val_loss: 4.1788 - val_acc: 0.7407
Epoch 8/10
10/10 [==============================] - 14s 1s/step - loss: 4.4883 - acc: 0.72

OSError: Unable to create file (Unable to open file: name = 'modelos/checkpoint.h5', errno = 2, error message = 'no such file or directory', flags = 13, o_flags = 302)

## Validação

In [58]:
from keras.preprocessing import image
from PIL import Image

diretorio_list = ['imagens/validacao/true/', 'imagens/validacao/false/']
for diretorio in diretorio_list:
    for img in glob.glob(diretorio + '*.png'):
        print(Image.open(img).filename[-18:-14])

        img = image.load_img(img, target_size=(256, 256))
        img_teste = image.img_to_array(img)
        img_teste = np.expand_dims(img_teste, axis=0) * 1./255
        prediction = model.predict(img_teste)
        
        print("Resultado: ", prediction);

0164
Resultado:  [[9.670172e-24]]
0169
Resultado:  [[6.0065226e-24]]
0179
Resultado:  [[1.0256557e-25]]
0181
Resultado:  [[6.2094616e-27]]
0183
Resultado:  [[3.1304208e-21]]
0184
Resultado:  [[9.823733e-23]]
0186
Resultado:  [[6.073041e-23]]
0161
Resultado:  [[8.471261e-25]]
0162
Resultado:  [[3.1370026e-24]]
0163
Resultado:  [[4.5950444e-23]]
0165
Resultado:  [[6.7217235e-25]]
0166
Resultado:  [[9.9987127e-29]]
0167
Resultado:  [[1.3158549e-15]]
0168
Resultado:  [[1.9386811e-27]]
0170
Resultado:  [[1.800421e-22]]
0171
Resultado:  [[1.7998077e-17]]
0172
Resultado:  [[2.5785978e-24]]
0173
Resultado:  [[3.1746113e-31]]
0174
Resultado:  [[1.9842317e-27]]
0175
Resultado:  [[1.3785829e-23]]
0176
Resultado:  [[8.101425e-17]]
0177
Resultado:  [[5.137979e-26]]
0178
Resultado:  [[2.0519278e-24]]
0180
Resultado:  [[1.0083562e-27]]
0182
Resultado:  [[4.2859526e-27]]
0184
Resultado:  [[9.823733e-23]]
0185
Resultado:  [[5.6429926e-24]]
